# Notebook displaying usage of the three main functionalities of *ritme*

## CLI example usage

````
python ../q2_ritme/split_train_test.py data_splits data/metadata_proc_v20240323_r0_r3_le_2yrs.tsv data/all_otu_table_filt.qza host_id --seed 12
````

````
python ../q2_ritme/find_best_model_config.py ../config/r_local_all.json data_splits/train_val.pkl --path-to-tax data/otu_taxonomy_all.qza --path-to-tree-phylo data/silva-138-99-rooted-tree.qza --path-store-model-logs ritme_refact_logs
````

````
python ../q2_ritme/evaluate_tuned_models.py ritme_refact_logs/r_local_all_mlruns data_splits/train_val.pkl data_splits/test.pkl
````

## Python API example usage

In [ ]:
from q2_ritme.find_best_model_config import (
    _load_experiment_config,
    _load_phylogeny,
    _load_taxonomy,
    find_best_model_config,
)
from q2_ritme.split_train_test import _load_data, split_train_test
from q2_ritme.evaluate_tuned_models import evaluate_tuned_models

%load_ext autoreload
%autoreload 2

In [ ]:
model_config_path = "../config/r_local_linreg.json"
config = _load_experiment_config(model_config_path)

### Read & split data

In [ ]:
md, ft = _load_data(config["path_to_md"], config["path_to_ft"])
print(md.shape, ft.shape)

In [ ]:
train_val, test = split_train_test(
    md,
    ft,
    stratify_by_column=config["stratify_by_column"],
    feature_prefix=config["feature_prefix"],
    train_size=config["train_size"],
    seed=config["seed_data"],
)

### Find best model config

In [ ]:
tax = _load_taxonomy(config["path_to_tax"])
phylo = _load_phylogeny(config["path_to_phylo"])

best_model_dict, path_to_exp = find_best_model_config(
    config, train_val, tax, phylo, path_store_model_logs="ritme_refact_logs"
)

### Evaluate best models

In [ ]:
metrics = evaluate_tuned_models(best_model_dict, config, train_val, test)
metrics